In [1]:
import os
import gc
import tensorflow as tf
import transformers
from transformers import TFBertForSequenceClassification, BertTokenizer

from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
import numpy as np
import pandas as pd
import pickle

import mlflow

2024-11-19 12:12:04.035759: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732018324.048966  559654 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732018324.052872  559654 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-19 12:12:04.066926: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
#mlflow.set_tracking_uri("http://localhost:5000")

In [3]:
# Affichage des versions pour vérification
print("TensorFlow version:", tf.__version__)
print("Transformers version:", transformers.__version__)

TensorFlow version: 2.18.0
Transformers version: 4.46.2


In [4]:
# Recharger le DataFrame depuis le fichier pickle
df_sample = pd.read_pickle('download/df_sample_20000.pkl')
df_sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   target  20000 non-null  int64 
 1   ids     20000 non-null  int64 
 2   date    20000 non-null  object
 3   flag    20000 non-null  object
 4   user    20000 non-null  object
 5   text    20000 non-null  object
dtypes: int64(2), object(4)
memory usage: 937.6+ KB


In [5]:
 #Vérifier si le DataFrame a au moins 16 000 lignes
if len(df_sample) != 20000:
    raise ValueError("Le DataFrame contient moins de 16 000 lignes.")

# # Calculer la proportion nécessaire pour obtenir 16 000 lignes
# sample_size = 16000 / len(df_sample)

# # Utiliser train_test_split pour sélectionner un échantillon équilibré de 16 000 lignes
# df_16000, _ = train_test_split(df_sample, train_size=sample_size, stratify=df_sample['target'], random_state=42)

# # Vérifier le nombre d'éléments et l'équilibre des classes
# print(f"Nombre d'échantillons conservés: {len(df_16000)}")
# print(df_16000['target'].value_counts(normalize=True))  # Vérifier l'équilibre des classes

display(df_sample['target'].value_counts())
df = df_sample

target
0    10000
1    10000
Name: count, dtype: int64

In [6]:
# Filtrer pour un sous-ensemble
# Taille de l'échantillon désiré
sample_size = 1000

# Calcul du nombre d'échantillons par classe
classes = df['target'].unique()
samples_per_class = sample_size // len(classes)

# Échantillonnage stratifié
df_sample = df.groupby('target').apply(lambda x: x.sample(n=samples_per_class)).reset_index(drop=True)
display(df_sample['target'].value_counts())

/tmp/ipykernel_559654/3865627404.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_sample = df.groupby('target').apply(lambda x: x.sample(n=samples_per_class)).reset_index(drop=True)


target
0    500
1    500
Name: count, dtype: int64

In [20]:
import re
import emoji
from transformers import AutoTokenizer, TFAutoModel, AutoConfig
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Input, Lambda
from tensorflow.keras.models import Model
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import mlflow
import mlflow.tensorflow

# Prétraitement des tweets
def preprocess_tweet(tweet):
    tweet = re.sub(r'http\S+|www\S+|https\S+', '', tweet)
    tweet = re.sub(r'@\w+', '', tweet)
    tweet = emoji.demojize(tweet)
    tweet = tweet.lower()
    tweet = re.sub(r'[^a-zA-Z0-9\s]', '', tweet)
    tweet = re.sub(r'\s+', ' ', tweet).strip()
    return tweet

# Nettoyage des tweets
# df_sample = df_sample.copy().sample(frac=0.1, random_state=42)  # Utilisez 10% des données
# display(df_sample['target'].value_counts())
df_sample['cleaned_text'] = df_sample['text'].apply(preprocess_tweet)

# Diviser les données en train et test
documents = df_sample['cleaned_text']
labels = df_sample['target']
X_train, X_val, y_train, y_val = train_test_split(documents, labels, test_size=0.2, random_state=42)

# Charger le tokenizer
model_name = 'finiteautomata/bertweet-base-sentiment-analysis'
#model_name = 'cardiffnlp/twitter-roberta-base-sentiment'
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenisation des données
def tokenize_texts(texts, max_length=128):
    return tokenizer(
        texts.tolist(),
        add_special_tokens=True,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_tensors="tf"
    )

train_encodings = tokenize_texts(X_train, max_length=80)
val_encodings = tokenize_texts(X_val, max_length=80)

# Préparer les datasets TensorFlow
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train.values
)).shuffle(len(X_train)).batch(16)

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    y_val.values
)).batch(16)

# Charger la configuration et le modèle pré-entraîné sans la couche de classification
config = AutoConfig.from_pretrained(model_name, output_hidden_states=False)
base_model = TFAutoModel.from_pretrained(model_name, config=config)

# Geler les 10 premières couches du modèle pour accélérer l'entraînement
# for layer in base_model.layers[:-1]:
#     layer.trainable = False
# base_model.layers[-1].trainable = True


# Créer un nouveau modèle Keras
input_ids = Input(shape=(80,), dtype=tf.int32, name="input_ids")
attention_mask = Input(shape=(80,), dtype=tf.int32, name="attention_mask")

# Utiliser Lambda pour appeler le modèle pré-entraîné avec output_shape spécifié
sequence_output = Lambda(
    lambda inputs: base_model(input_ids=inputs[0], attention_mask=inputs[1])[0],
    output_shape=(80, 768)
)([input_ids, attention_mask])

cls_token = Lambda(lambda x: x[:, 0, :], output_shape=(768,))(sequence_output)  # Extraire le token [CLS]

# Ajouter une couche dense pour la classification binaire
output = Dense(1, activation="sigmoid")(cls_token)

# Construire le modèle final
model = Model(inputs=[input_ids, attention_mask], outputs=output)

# Optimiseur et perte
learning_rate = 1e-5
optimizer = Adam()
loss = tf.keras.losses.BinaryCrossentropy(from_logits=False)
metric = tf.keras.metrics.BinaryAccuracy('accuracy')

# Compiler le modèle
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

# Intégration avec MLflow
#mlflow.tensorflow.autolog()

with mlflow.start_run(run_name=f"Fine-tuning {model_name}", nested=True):
    mlflow.set_tag("model_name", model_name)
    mlflow.log_param("epochs", 5)
    mlflow.log_param("batch_size", 16)
    mlflow.log_param("learning_rate", learning_rate)

    # Désactiver l'autolog de TensorFlow
    #mlflow.tensorflow.autolog(disable=True)

    # Entraîner le modèle
    history = model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=5
    )

    # Prédictions sur le jeu de validation
    val_logits = model.predict(val_dataset)
    y_val_pred = (val_logits > 0.5).astype(int)

    # Évaluer les performances
    accuracy = accuracy_score(y_val, y_val_pred)
    print("Validation Accuracy:", accuracy)
    print("Classification Report:")
    print(classification_report(y_val, y_val_pred))

    # # Log des métriques finales dans MLflow
    # mlflow.log_metric("Validation Accuracy", accuracy)
    # mlflow.log_dict(
    #     classification_report(y_val, y_val_pred, output_dict=True),
    #     "classification_report.json"
    # )


Some layers from the model checkpoint at finiteautomata/bertweet-base-sentiment-analysis were not used when initializing TFRobertaModel: ['classifier']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFRobertaModel were not initialized from the model checkpoint at finiteautomata/bertweet-base-sentiment-analysis and are newly initialized: ['roberta/pooler/dense/bias:0', 'roberta/pooler/dense/kernel:0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2024-11-19 13:12:26.843461: I tensorflow/core/framewor

Epoch 1/5


/opt/venv/lib/python3.11/site-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['input_ids', 'attention_mask']. Received: the structure of inputs={'input_ids': '*', 'token_type_ids': '*', 'attention_mask': '*'}
  warnings.warn(


50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 684ms/step - accuracy: 0.4953 - loss: 0.6986

50/50 ━━━━━━━━━━━━━━━━━━━━ 52s 892ms/step - accuracy: 0.4953 - loss: 0.6986 - val_accuracy: 0.4800 - val_loss: 0.7189
Epoch 2/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 662ms/step - accuracy: 0.5053 - loss: 0.7028

...

50/50 ━━━━━━━━━━━━━━━━━━━━ 41s 824ms/step - accuracy: 0.4785 - loss: 0.7052 - val_accuracy: 0.5200 - val_loss: 0.6924
Epoch 5/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 41s 823ms/step - accuracy: 0.5102 - loss: 0.6945 - val_accuracy: 0.4800 - val_loss: 0.6946
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


2024/11/19 13:16:09 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmp9xabzx03/model, flavor: tensorflow). Fall back to return ['tensorflow==2.18.0', 'cloudpickle==3.1.0']. Set logging level to DEBUG to see the full traceback. 


13/13 ━━━━━━━━━━━━━━━━━━━━ 11s 782ms/step


/opt/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
2024/11/19 13:16:20 INFO mlflow.tracking._tracking_s

Validation Accuracy: 0.48
Classification Report:
              precision    recall  f1-score   support

           0       0.48      1.00      0.65        96
           1       0.00      0.00      0.00       104

    accuracy                           0.48       200
   macro avg       0.24      0.50      0.32       200
weighted avg       0.23      0.48      0.31       200



In [23]:
display(df_sample['target'].value_counts())


target
0    500
1    500
Name: count, dtype: int64

## Annexe

#Annexe

In [24]:

# Charger le tokenizer et le modèle
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [25]:
# Exemple de données pour 'documents' et 'labels'
documents = df_sample['text']  # Liste de textes à analyser
labels = df_sample['target']   # Liste de labels (0 ou 1 pour la classification binaire)


In [26]:
# Tokenization des données
def tokenize_data(documents):
    return tokenizer(
        documents.tolist(),
        max_length=128, padding=True, truncation=True, return_tensors='tf'
    )
tokens = tokenize_data(documents)

In [27]:
# Convertir en NumPy pour train_test_split
input_ids_np = tokens['input_ids'].numpy()
attention_masks_np = tokens['attention_mask'].numpy()
labels_np = np.array(labels)

In [28]:
# Séparer les ensembles de données
train_input_ids, val_input_ids, train_labels, val_labels = train_test_split(input_ids_np, labels_np, test_size=0.2, random_state=42)
train_attention_masks, val_attention_masks = train_test_split(attention_masks_np, test_size=0.2, random_state=42)


In [29]:
# Configurer la fonction de perte et l'optimiseur
loss_fn = SparseCategoricalCrossentropy(from_logits=True)
optimizer = Adam(learning_rate=1e-5)

In [30]:
# Définir le nombre d'époques et la taille du batch
epochs = 5
batch_size = 16

# Définir l'accuracy
train_accuracy_metric = tf.keras.metrics.SparseCategoricalAccuracy()

In [31]:
# Configuration MLflow
experiment = mlflow.set_experiment("BERT")

with mlflow.start_run(run_name="BERT", nested=True):
    # Enregistrement des hyperparamètres
    mlflow.log_param("model_name", "bert-base-uncased")
    mlflow.log_param("num_epochs", epochs)
    mlflow.log_param("batch_size", batch_size)
    mlflow.log_param("learning_rate", 1e-5)
    
    # Boucle d'entraînement
    for epoch in range(epochs):
        print(f"\nEpoch {epoch + 1}/{epochs}")
        train_accuracy_metric.reset_state()  # Réinitialiser l'accuracy pour chaque epoch
        epoch_loss = []  # Réinitialiser la liste des pertes pour chaque epoch

        for i in range(0, len(train_input_ids), batch_size):
            # Obtenir un batch de données
            batch_input_ids = train_input_ids[i:i + batch_size]
            batch_attention_masks = train_attention_masks[i:i + batch_size]
            batch_labels = train_labels[i:i + batch_size]
            
            with tf.GradientTape() as tape:
                # Faire des prédictions
                outputs = model(
                    input_ids=batch_input_ids,
                    attention_mask=batch_attention_masks,
                    training=True
                )
                logits = outputs.logits
                
                # Calculer la perte
                loss = loss_fn(batch_labels, logits)
            
            # Calculer et appliquer les gradients
            grads = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))
            
            # Mettre à jour l'accuracy et accumuler la perte
            train_accuracy_metric.update_state(batch_labels, logits)
            epoch_loss.append(loss.numpy())
            
            # Afficher la perte et l'accuracy toutes les 10 batches
            if i % (batch_size * 10) == 0:
                train_accuracy = train_accuracy_metric.result().numpy()
                print(f"Batch {i//batch_size} - Loss: {loss.numpy():.4f} - Accuracy: {train_accuracy:.4f}")
                mlflow.log_metric("batch_train_loss", loss.numpy(), step=i//batch_size)
                mlflow.log_metric("batch_train_accuracy", train_accuracy, step=i//batch_size)
        
        # Enregistrement des métriques de l'époque
        epoch_accuracy = train_accuracy_metric.result().numpy()
        epoch_loss_avg = np.mean(epoch_loss)
        print(f"Epoch {epoch + 1} - Loss: {epoch_loss_avg:.4f} - Accuracy: {epoch_accuracy:.4f}")
        mlflow.log_metric("epoch_train_loss", epoch_loss_avg, step=epoch)
        mlflow.log_metric("epoch_train_accuracy", epoch_accuracy, step=epoch)

    # Libérer la mémoire GPU/CPU non utilisée avant l'évaluation
    gc.collect()
    tf.keras.backend.clear_session()


Epoch 1/5
Batch 0 - Loss: 0.7571 - Accuracy: 0.3750
Batch 10 - Loss: 0.6762 - Accuracy: 0.5852
Batch 20 - Loss: 0.6756 - Accuracy: 0.6012
Batch 30 - Loss: 0.7363 - Accuracy: 0.6129
Batch 40 - Loss: 0.6564 - Accuracy: 0.6174
Epoch 1 - Loss: 0.6640 - Accuracy: 0.6175

Epoch 2/5
Batch 0 - Loss: 0.6280 - Accuracy: 0.6875
Batch 10 - Loss: 0.5860 - Accuracy: 0.7614
Batch 20 - Loss: 0.5801 - Accuracy: 0.7381
Batch 30 - Loss: 0.5177 - Accuracy: 0.7520
Batch 40 - Loss: 0.5065 - Accuracy: 0.7637
Epoch 2 - Loss: 0.5328 - Accuracy: 0.7763

Epoch 3/5
Batch 0 - Loss: 0.3318 - Accuracy: 0.9375
Batch 10 - Loss: 0.3797 - Accuracy: 0.8693
Batch 20 - Loss: 0.3074 - Accuracy: 0.8690
Batch 30 - Loss: 0.2170 - Accuracy: 0.8669
Batch 40 - Loss: 0.2157 - Accuracy: 0.8750
Epoch 3 - Loss: 0.3613 - Accuracy: 0.8788

Epoch 4/5
Batch 0 - Loss: 0.1782 - Accuracy: 1.0000
Batch 10 - Loss: 0.3427 - Accuracy: 0.9318
Batch 20 - Loss: 0.1576 - Accuracy: 0.9405
Batch 30 - Loss: 0.1141 - Accuracy: 0.9435
Batch 40 - Loss: 

2024/11/19 13:58:25 INFO mlflow.tracking._tracking_service.client: 🏃 View run BERT at: http://mlflow-server:5000/#/experiments/6/runs/7f74236345d4450d808e78ef5ccde02c.
2024/11/19 13:58:25 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://mlflow-server:5000/#/experiments/6.


In [32]:
from io import BytesIO

def log_plot_to_mlflow(figure, artifact_name):
    """
    Enregistre une figure matplotlib dans MLflow en utilisant un buffer en mémoire.

    :param figure: la figure matplotlib à sauvegarder
    :param artifact_name: le nom de l'artefact pour MLflow (inclure ".png")
    """
    # Utilisation d'un buffer en mémoire
    buffer = BytesIO()
    figure.savefig(buffer, format="png")
    buffer.seek(0)
    
    # Sauvegarder dans MLflow à partir du buffer
    with open(artifact_name, "wb") as f:
        f.write(buffer.getvalue())
    mlflow.log_artifact(artifact_name)

In [33]:
 # Calcul et stockage des métriques dans un dictionnaire
from sklearn.metrics import precision_score, recall_score, f1_score, roc_curve, auc
    
# Évaluation du modèle sur le set de validation en mini-batches
batch_size = 8  # Taille réduite pour l'évaluation
num_batches = len(val_input_ids) // batch_size
all_predictions = []
# Initialisation du dictionnaire des métriques
metrics_dict = {}
all_probs = []

for i in range(num_batches):
    # Obtenir un batch de validation
    batch_input_ids = val_input_ids[i * batch_size : (i + 1) * batch_size]
    batch_attention_masks = val_attention_masks[i * batch_size : (i + 1) * batch_size]

    # Calcul des logits pour le batch
    batch_logits = model(
        input_ids=batch_input_ids,
        attention_mask=batch_attention_masks,
        training=False
    ).logits

    # Stocker les prédictions
    batch_predictions = tf.argmax(batch_logits, axis=1)
    all_predictions.append(batch_predictions)
    
     # Convertir logits en probabilités
    batch_probabilities = tf.nn.softmax(batch_logits, axis=1)[:, 1]  # Probabilités pour la classe 1
    all_probs.append(batch_probabilities)

# Concaténer toutes les prédictions
all_predictions = tf.concat(all_predictions, axis=0)

# Concaténer toutes les probabilités
all_probs = tf.concat(all_probs, axis=0).numpy()

# Calcul de la courbe ROC
fpr, tpr, thresholds = roc_curve(val_labels[:len(all_probs)], all_probs)
roc_auc = auc(fpr, tpr)

# Tracer la courbe ROC
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f"ROC curve (AUC = {roc_auc:.2f})")
plt.plot([0, 1], [0, 1], 'k--', label="Random guess")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Bert ROC Curve - Validation Set")
plt.legend(loc="lower right")
log_plot_to_mlflow(plt.gcf(), "roc_curve_val.png")
plt.close()



# Calcul de l'accuracy
accuracy = tf.reduce_mean(tf.cast(all_predictions == val_labels[:len(all_predictions)], dtype=tf.float32))
print(f"Validation Accuracy: {accuracy.numpy():.4f}")

validation_metrics = {
    "Validation Accuracy": round(float(accuracy.numpy()), 3),
    "Validation ROC AUC": round(roc_auc, 3),
    "Validation Precision": round(precision_score(val_labels[:len(all_predictions)], all_predictions.numpy()), 3),
    "Validation Recall": round(recall_score(val_labels[:len(all_predictions)], all_predictions.numpy()), 3),
    "Validation F1": round(f1_score(val_labels[:len(all_predictions)], all_predictions.numpy()), 3)
}

metrics_dict.update(validation_metrics)
# Log de l'accuracy finale de validation et des autres métriques dans MLflow
mlflow.log_metrics(metrics_dict)

# Log de l'accuracy finale de validation dans MLflow
mlflow.log_metric("val_accuracy", accuracy.numpy())

# Enregistrement du modèle
mlflow.keras.log_model(model, "bert_model")

run_id = mlflow.active_run().info.run_id
result = mlflow.register_model(
    model_uri=f"runs:/{run_id}/model",
    name=f"bert-base-uncased"
)

2024/11/19 13:58:45 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.


Validation Accuracy: 0.7500


/opt/venv/lib/python3.11/site-packages/transformers/generation/tf_utils.py:465: UserWarning: `seed_generator` is deprecated and will be removed in a future version.
  warnings.warn("`seed_generator` is deprecated and will be removed in a future version.", UserWarning)
2024/11/19 13:58:51 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmph_8ytx4l/model, flavor: keras). Fall back to return ['keras==3.6.0']. Set logging level to DEBUG to see the full traceback. 
2024/11/19 13:58:51 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'bert-base-uncased' already exists. Creating a new version of this model...
2024/11/19 13:58:53 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: bert-base-uncased, version 3
Crea

In [34]:
display(validation_metrics)

{'Validation Accuracy': 0.75,
 'Validation ROC AUC': 0.875,
 'Validation Precision': 0.821,
 'Validation Recall': 0.663,
 'Validation F1': 0.734}

In [35]:
display(pd.DataFrame(metrics_dict, index=[0]))

,Validation Accuracy,Validation ROC AUC,Validation Precision,Validation Recall,Validation F1
0,0.75,0.875,0.821,0.663,0.734
